In [1]:
from google.colab import drive 
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
%cd /content/gdrive/My\ Drive/bbbc-005-dataset

/content/gdrive/My Drive/bbbc-005-dataset


In [3]:
!ls

BBBC005_cells_dataset_final.tar.xz  cell-counting-model-training.ipynb
BBBC005_cells_dataset_final.zip     generator.py


In [0]:
!tar -xvf  BBBC005_cells_dataset_final.tar.xz

In [5]:
!ls

BBBC005_cells_dataset_final	    cell-counting-model-training.ipynb
BBBC005_cells_dataset_final.tar.xz  generator.py
BBBC005_cells_dataset_final.zip


In [6]:
# import statements for constructing the model design.
from __future__ import absolute_import
from __future__ import print_function
import numpy as np
from keras import backend as K
from keras.models import Sequential, Model
from keras.layers import (
    Input,
    Activation,
    concatenate,
    add,
    merge,
    Dropout,
    Reshape,
    Permute,
    Dense,
    UpSampling2D,
    Flatten
    )
from keras.optimizers import SGD
from keras.layers.convolutional import (
    Convolution2D)
from keras.layers.pooling import (
    MaxPooling2D,
    AveragePooling2D
    )
from keras.layers.normalization import BatchNormalization
from keras.regularizers import l2
import numpy as np
import keras.backend as K
from keras.layers import *
from keras.initializers import *
from keras.models import *
from keras import initializers

Using TensorFlow backend.


In [0]:
# Code is referenced from: [kgrm](https://github.com/kgrm)
class NALU(Layer):
    def __init__(self, units, MW_initializer='glorot_uniform',
                 G_initializer='glorot_uniform', mode="NALU",
                 **kwargs):
        if 'input_shape' not in kwargs and 'input_dim' in kwargs:
            kwargs['input_shape'] = (kwargs.pop('input_dim'),)
        super(NALU, self).__init__(**kwargs)
        self.units = units
        self.mode = mode
        self.MW_initializer = initializers.get(MW_initializer)
        self.G_initializer = initializers.get(G_initializer)
        self.input_spec = InputSpec(min_ndim=2)
        self.supports_masking = True

    def build(self, input_shape):
        assert len(input_shape) >= 2
        input_dim = input_shape[-1]

        self.W_hat = self.add_weight(shape=(input_dim, self.units),
                                     initializer=self.MW_initializer,
                                     name='W_hat')
        self.M_hat = self.add_weight(shape=(input_dim, self.units),
                                     initializer=self.MW_initializer,
                                     name='M_hat')
        if self.mode == "NALU":
            self.G = self.add_weight(shape=(input_dim, self.units),
                                     initializer=self.G_initializer,
                                     name='G')
        self.input_spec = InputSpec(min_ndim=2, axes={-1: input_dim})
        self.built = True

    def call(self, inputs):
        W = K.tanh(self.W_hat) * K.sigmoid(self.M_hat)
        a = K.dot(inputs, W)
        if self.mode == "NAC":
            output = a
        elif self.mode == "NALU":
            m = K.exp(K.dot(K.log(K.abs(inputs) + 1e-7), W))
            g = K.sigmoid(K.dot(K.abs(inputs), self.G))
            output = g * a + (1 - g) * m
        else:
            raise ValueError("Valid modes: 'NAC', 'NALU'.")
        return output

    def compute_output_shape(self, input_shape):
        assert input_shape and len(input_shape) >= 2
        assert input_shape[-1]
        output_shape = list(input_shape)
        output_shape[-1] = self.units
        return tuple(output_shape)

In [0]:
# weight decay defined on model's parametric matrices for regularization purpose.
weight_decay = 1e-5

In [0]:
def _conv_bn_relu(nb_filter, row, col, subsample = (1,1)):
    def f(input):
        conv_a = Convolution2D(nb_filter, row, col, subsample = subsample,
                               init = 'orthogonal', 
                               border_mode='same', bias = False)(input)
        norm_a = BatchNormalization()(conv_a)
        # 1. This actionvation function can be changed as demonstrated in experiments to create FCRN/U-net variants.
        act_a = Activation(activation = 'relu')(norm_a)
        return act_a
    return f

In [0]:
def _conv_bn_relu_x2(nb_filter, row, col, subsample = (1,1)):
    def f(input):
        # 1. batch normalization with relu activation, again it can be changed to create different variants of FCRN & U-net.
        conv_a = Convolution2D(nb_filter, row, col, subsample = subsample,
                               init = 'orthogonal', border_mode = 'same',bias = False,
                               W_regularizer = l2(weight_decay),
                               b_regularizer = l2(weight_decay))(input)
        norm_a = BatchNormalization()(conv_a)
        act_a = Activation(activation = 'relu')(norm_a)
        conv_b = Convolution2D(nb_filter, row, col, subsample = subsample,
                              init = 'orthogonal', border_mode = 'same',bias = False,
                              W_regularizer = l2(weight_decay),
                              b_regularizer = l2(weight_decay))(act_a)
        norm_b = BatchNormalization()(conv_b)
        act_b = Activation(activation = 'relu')(norm_b)
        return act_a
    return f

In [0]:
def U_net_base(input, nb_filter = 64):
    block1 = _conv_bn_relu_x2(nb_filter,3,3)(input)
    pool1 = MaxPooling2D(pool_size=(2,2))(block1)
    # =========================================================================
    block2 = _conv_bn_relu_x2(128,3,3)(pool1)
    pool2 = MaxPooling2D(pool_size=(2, 2))(block2)
    # =========================================================================
    block3 = _conv_bn_relu_x2(256,3,3)(pool2)
    pool3 = MaxPooling2D(pool_size=(2, 2))(block3)
    # =========================================================================
    block4 = _conv_bn_relu_x2(256,3,3)(pool3)
    up4 = merge([UpSampling2D(size=(2, 2))(block4), block3], mode='concat', concat_axis=-1)
    # =========================================================================
    block5 = _conv_bn_relu_x2(128,3,3)(up4)
    up5 = merge([UpSampling2D(size=(2, 2))(block5), block2], mode='concat', concat_axis=-1)
    # =========================================================================
    block6 = _conv_bn_relu_x2(nb_filter,3,3)(up5)
    up6 = merge([UpSampling2D(size=(2, 2))(block6), block1], mode='concat', concat_axis=-1)
    # =========================================================================
    block7 = _conv_bn_relu(32,3,3)(up6)
    return block7

def u_net_nalu(input, nb_filter = 64):
    # input-(256,256,3), output-(128,128,64)
    block1 = _conv_bn_relu_x2(64,3,3)(input)
    pool1 = MaxPooling2D(pool_size=(2,2))(block1)
    nal1 = NALU(64, mode="NAC", 
             MW_initializer=RandomNormal(stddev=1),
             G_initializer=Constant(10))(block1) # volume- (256,256,64)
    # =========================================================================
    # input-(128,128,64), output-(64,64,128)
    block2 = _conv_bn_relu_x2(128,3,3)(pool1)
    pool2 = MaxPooling2D(pool_size=(2, 2))(block2)
    nal2 = NALU(128, mode="NAC", 
             MW_initializer=RandomNormal(stddev=1),
             G_initializer=Constant(10))(block2) # volume- (128,128,128)
    # ========================================================================= 
    # input-(64,64,128), output-(32,32,256)
    block3 = _conv_bn_relu_x2(256,3,3)(pool2)
    pool3 = MaxPooling2D(pool_size=(2, 2))(block3)
    nal3 = NALU(256, mode="NAC", 
             MW_initializer=RandomNormal(stddev=1),
             G_initializer=Constant(10))(block3) # volume- (64,64,256)
    # =========================================================================
    # input-(32,32,256), output-(64,64,256)
    block4 = _conv_bn_relu_x2(256,3,3)(pool3)
    up4 = concatenate([UpSampling2D(size=(2, 2))(block4), block3], axis=-1)
    up4 = concatenate([up4, nal3], axis=-1)
    up4 = _conv_bn_relu_x2(256,3,3)(up4)
    # =========================================================================
    # input-(64,64,256), output-(128,128,128)
    block5 = _conv_bn_relu_x2(128,3,3)(up4)
    up5 = concatenate([UpSampling2D(size=(2, 2))(block5), block2], axis=-1)
    up5 = concatenate([up5, nal2], axis=-1)
    up5 = _conv_bn_relu_x2(128,3,3)(up5)
    # =========================================================================
    # input-(128,128,128), output-(256,256,64)
    block6 = _conv_bn_relu_x2(64,3,3)(up5)
    up6 = concatenate([UpSampling2D(size=(2, 2))(block6), block1], axis=-1) # input-128, output-256
    up6 = concatenate([up6, nal1], axis=-1)
    up6 = _conv_bn_relu_x2(64,3,3)(up6)
    # =========================================================================
    # input-(256,256,64), output-(256,256,32)
    block7 = _conv_bn_relu(32,3,3)(up6)
    return block7

In [0]:
def buildmodel_u_net (input_dim):
    input_ = Input (shape = (input_dim))
    # =========================================================================
    # Change the base here to train different model base structure.
    act_ = U_net_base (input_, nb_filter = 64 )
    # =========================================================================
    density_pred =  Convolution2D(1, 1, 1, bias = False, activation='linear',\
                                  init='orthogonal',name='pred',border_mode='same')(act_)
    # =========================================================================
    model = Model (input = input_, output = density_pred)
    opt = SGD(lr = 1e-2, momentum = 0.9, nesterov = True)
    model.compile(optimizer = opt, loss = 'mse')
    return model

In [0]:
def buildmodel_u_net_nalu (input_dim):
    input_ = Input (shape = (input_dim))
    # =========================================================================
    # Change the base here to train different model base structure.
    act_ = u_net_nalu (input_, nb_filter = 64 )
    # =========================================================================
    density_pred =  Convolution2D(1, 1, 1, bias = False, activation='linear',\
                                  init='orthogonal',name='pred',border_mode='same')(act_)
    # =========================================================================
    model = Model (input = input_, output = density_pred)
    opt = SGD(lr = 1e-2, momentum = 0.9, nesterov = True)
    model.compile(optimizer = opt, loss = 'mse')
    return model

In [0]:
from generator import ImageDataGenerator

In [0]:
import numpy as np
import pdb
import os
import matplotlib.pyplot as plt
from keras import backend as K
from keras.callbacks import ModelCheckpoint,Callback,LearningRateScheduler
from scipy import misc
import imageio
import scipy.ndimage as ndimage

class LossHistory(Callback):
    def on_train_begin(self, logs={}):
        self.losses = []

    def on_batch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))

base_path = 'BBBC005_cells_dataset_final/'
data = []
anno = []

def step_decay(epoch):
    # step decay implementation.
    step = 16
    num =  epoch // step 
    if num % 3 == 0:
        lrate = 1e-3
    elif num % 3 == 1:
        lrate = 1e-4
    else:
        lrate = 1e-5

        # continous decaying implementation. decay lrate first time and then assign new learning with every 
        # initial_lrate = lrate
        # lrate = initial_lrate * 1/(1 + decay * (epoch - num * step))
    print('Learning rate for epoch {} is {}.'.format(epoch+1, lrate))    
    return np.float(lrate)

def read_data(base_path):
    imList = os.listdir(base_path)
    for i in range(len(imList)): 
        if 'cell' in imList[i]:
            img1 = imageio.imread(os.path.join(base_path,imList[i]))
            data.append(img1)
            
            img2_ = imageio.imread(os.path.join(base_path, imList[i][:4] + 'dots.png'))
            # amplify the dot annotation values in R channel of RGB.
            # Omit this amplification step if you are trying to train this model on other images.
            # BBBC005 dataset doesn't require amplitude increment for any pixel value like flourescent synthetic dataset.
            img2_ = 1.0 * (img2_[:,:,0] > 0)
            img2 = ndimage.gaussian_filter(img2_, sigma=(1, 1), order=0)
            anno.append(img2)
    return np.asarray(data, dtype = 'float32'), np.asarray(anno, dtype = 'float32')


def train_(base_path):
    data, anno = read_data(base_path)
    anno = np.expand_dims(anno, axis = -1)
    
    mean = np.mean(data)
    std = np.std(data)
    
    data_ = (data - mean) / std
    
    train_data = data_[:1470]
    train_anno = anno[:1470]

    val_data = data_[1470:]
    val_anno = anno[1470:]
    
    print('-'*30)
    print('Creating and compiling the fully convolutional regression networks.')
    print('-'*30)    
   
    # model building step: change this line to build new model each time.
    model = buildmodel_u_net_nalu(input_dim = (256,256,3))
    # model = buildmodel_u_net(input_dim = (256,256,3))

    # Also, change this line for saving a new variant of a model while training.
    # Here, a demo model is just executed for getting started.
    model_checkpoint = ModelCheckpoint('cell_counting_unet_nalu.hdf5', monitor='loss', save_best_only=True)
    model.summary()
    print('...Fitting model...')
    print('-'*30)
    change_lr = LearningRateScheduler(step_decay)

    datagen = ImageDataGenerator(
        featurewise_center = False,  # set input mean to 0 over the dataset
        samplewise_center = False,  # set each sample mean to 0
        featurewise_std_normalization = False,  # divide inputs by std of the dataset
        samplewise_std_normalization = False,  # divide each input by its std
        zca_whitening = False,  # apply ZCA whitening
        rotation_range = 30,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range = 0.3,  # randomly shift images horizontally (fraction of total width)
        height_shift_range = 0.3,  # randomly shift images vertically (fraction of total height)
        zoom_range = 0.3,
        shear_range = 0.,
        horizontal_flip = True,  # randomly flip images
        vertical_flip = True, # randomly flip images
        fill_mode = 'constant',
        dim_ordering = 'tf')  

    # list of epochs batches to measure MAE after training & stopping the training after appropriate epochs and avoiding overfitting.
    # Assumption for this MAE measurement approach is that model definitely achieves ideal prediction after 64 epochs.
    # An epoch offset list logic to determine models performance at best and stopping that point.
    lst_epochs = [64,16,8,16,8,16,8,16,8,16,8,16,8,16,8,8,8,8,8,8,4,4,2,2]
    # Global error value to capture the minimum most error value on test set.
    global_err_val = 100.0
    # This measures offset value of epoch in which the training has to be stopped.
    ep_offset = 0
    for epoch in lst_epochs:
        ep_offset = epoch # assuming epochs required for training is greater than 64 always. A reasonable assumption for this dataset.
        model.fit_generator(datagen.flow(train_data,
                                        train_anno,
                                        batch_size = 16
                                        ),
                            samples_per_epoch = train_data.shape[0],
                            nb_epoch = epoch,
                            callbacks = [model_checkpoint, change_lr],
                        )
    
        model.load_weights('cell_counting_unet_nalu.hdf5')
        A = model.predict(val_data)    
        mean_diff = np.average(  np.abs(  np.sum(np.sum(A,1),1)-np.sum(np.sum(val_anno,1),1)  )  ) / (630.0)
        if mean_diff < global_err_val:
            global_err_val = mean_diff
        else:
            break

    print('After training, the mean absolute error is : {} cells per image.'.format(np.abs(global_err_val)))
    print('This error is obtained after training for: '+str(64+int(ep_offset)) + ' epochs' )

In [19]:
if __name__ == '__main__':
    train_(base_path)

------------------------------
Creating and compiling the fully convolutional regression networks.
------------------------------


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), strides=(1, 1), padding="same", kernel_initializer="orthogonal", kernel_regularizer=<keras.reg..., bias_regularizer=<keras.reg..., use_bias=False)`
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), strides=(1, 1), padding="same", kernel_initializer="orthogonal", kernel_regularizer=<keras.reg..., bias_regularizer=<keras.reg..., use_bias=False)`
  del sys.path[0]
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), strides=(1, 1), padding="same", kernel_initializer="orthogonal", kernel_regularizer=<keras.reg..., bias_regularizer=<keras.reg..., use_bias=False)`
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: UserWarning: Upd

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
conv2d_20 (Conv2D)              (None, 256, 256, 64) 1728        input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_20 (BatchNo (None, 256, 256, 64) 256         conv2d_20[0][0]                  
__________________________________________________________________________________________________
activation_20 (Activation)      (None, 256, 256, 64) 0           batch_normalization_20[0][0]     
____________________________________________________________________________________________

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:119: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., callbacks=[<keras.ca..., steps_per_epoch=91, epochs=16)`


Epoch 1/16
Learning rate for epoch 1 is 0.001.
91/91 [==============================] - 38s 423ms/step - loss: 0.0279
Epoch 2/16
Learning rate for epoch 2 is 0.001.
91/91 [==============================] - 38s 421ms/step - loss: 0.0278
Epoch 3/16
Learning rate for epoch 3 is 0.001.
91/91 [==============================] - 38s 421ms/step - loss: 0.0275
Epoch 4/16
Learning rate for epoch 4 is 0.001.
91/91 [==============================] - 38s 421ms/step - loss: 0.0274
Epoch 5/16
Learning rate for epoch 5 is 0.001.
91/91 [==============================] - 38s 421ms/step - loss: 0.0282
Epoch 6/16
Learning rate for epoch 6 is 0.001.
91/91 [==============================] - 38s 421ms/step - loss: 0.0275
Epoch 7/16
Learning rate for epoch 7 is 0.001.
91/91 [==============================] - 38s 421ms/step - loss: 0.0268
Epoch 8/16
Learning rate for epoch 8 is 0.001.
91/91 [==============================] - 38s 421ms/step - loss: 0.0277
Epoch 9/16
Learning rate for epoch 9 is 0.001.
91/91 [==

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:119: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., callbacks=[<keras.ca..., steps_per_epoch=91, epochs=8)`


Epoch 1/8
Learning rate for epoch 1 is 0.001.
91/91 [==============================] - 39s 424ms/step - loss: 0.0273
Epoch 2/8
Learning rate for epoch 2 is 0.001.
91/91 [==============================] - 38s 421ms/step - loss: 0.0275
Epoch 3/8
Learning rate for epoch 3 is 0.001.
91/91 [==============================] - 38s 421ms/step - loss: 0.0271
Epoch 4/8
Learning rate for epoch 4 is 0.001.
91/91 [==============================] - 38s 421ms/step - loss: 0.0275
Epoch 5/8
Learning rate for epoch 5 is 0.001.
91/91 [==============================] - 38s 421ms/step - loss: 0.0273
Epoch 6/8
Learning rate for epoch 6 is 0.001.
91/91 [==============================] - 38s 421ms/step - loss: 0.0272
Epoch 7/8
Learning rate for epoch 7 is 0.001.
91/91 [==============================] - 38s 421ms/step - loss: 0.0271
Epoch 8/8
Learning rate for epoch 8 is 0.001.
91/91 [==============================] - 38s 421ms/step - loss: 0.0274
After training, the mean absolute error is : 3.9764989459325397 

In [0]:
# Final Result for U-net NALU based model is stated for this dataset.
# ...
# Learning rate for epoch 8 is 0.001.
# 91/91 [==============================] - 38s 421ms/step - loss: 0.0274
# After training, the mean absolute error is : 3.9764989459325397 cells per image.
# This error is obtained after training for: 72 epochs